In [1]:
import numpy as np 
import pandas as pd 


import os
print(os.listdir("."))


from datetime import datetime,timedelta
from pytz import timezone
import pytz
from haversine import haversine
import gc
import time
import pickle
from multiprocessing import Pool
from numba import njit
import pygeohash as pgh
from sklearn import preprocessing
from dateutil import tz

['quantumblack_71_4.ipynb', 'quantumblack (3).ipynb', 'quantumblack (5).ipynb', 'saved_files', 'quantumblack.ipynb', 'quantumblack_71_3.ipynb', 'quantumblack_71_current.ipynb', 'all_data.pkl', 'quantumblack_71-Copy2.ipynb', 'engine_features.csv', 'quantumblack_71-5.ipynb', 'weather_features.csv', 'quantumblack_71-mean.ipynb', 'data', '.ipynb_checkpoints', 'drive_features.csv', 'quantumblack_71_current-Copy1.ipynb', 'quantumblack_71.ipynb']


In [2]:
# !rm -rf saved_files

In [3]:
start_time = time.time()

# Getting data and some EDA

In [4]:
input_path = "./data/"
sub_path = './data/'

In [5]:
if not os.path.exists('saved_files'):
    os.makedirs('saved_files')

In [6]:
def load_file(input_path,filename):
    df = None
    try:
        saved_file = 'saved_files/'+filename+'.pkl'
        df = pd.read_pickle(saved_file)
    except Exception as e:
        print('Exception ',e,'Could not load ','saved_files/'+filename+'.pkl',' making it now')
        df = pd.read_csv(input_path+filename+'.csv')
    with open('saved_files/'+filename+'.pkl',mode='wb') as saved_file:
        pickle.dump(df,saved_file)
    return df

def load_folder(input_path,folder_name):
    df = None
    try:
        saved_file = 'saved_files/'+folder_name+'.pkl'
        df = pd.read_pickle(saved_file)
    except Exception as e:
        print('Exception ',e,'Could not load ','saved_files/'+folder_name+'.pkl',' making it now')
        df = None
        for file in os.listdir(input_path + folder_name):
            if file.endswith('.parquet'):
                temp = pd.read_parquet(input_path + folder_name+'/'+str(file),engine ='pyarrow')
                if df is None:
                    df = temp
                else:
                    df = pd.concat([df,temp])
                    
    with open('saved_files/'+folder_name+'.pkl',mode='wb') as saved_file:
        pickle.dump(df,saved_file)
    return df

def downcast_dtypes(df):
    int_cols = [c for c in df if df[c].dtype == 'int64']
    float_cols = [c for c in df if df[c].dtype == 'float64']
    df[int_cols] = df[int_cols].astype(np.int32)
    df[float_cols] = df[float_cols].astype(np.float32)
    return df

def round_floats(df):
    float_cols = [c for c in df if df[c].dtype == 'float64']
    df[float_cols] = df[float_cols].round(0).astype(np.int32)
    float_cols = [c for c in df if df[c].dtype == 'float32']
    df[float_cols] = df[float_cols].round(0).astype(np.int32)
    return df


In [7]:
df = load_file(sub_path,'drive_features')
ef = load_file(sub_path,'engine_features')
wf = load_file(sub_path,'weather_features')
vehicle_df = load_file(input_path,'vehicle')
drive_df = load_folder(input_path,'drive')
trip_df = load_folder(input_path,'trip')
weather_df = load_folder(input_path,'weather')

In [8]:
vehicle_df.head(10)

,vehicle_id,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
0,1000500,2016,Honda,Civic,2,174,140,6500,1500,1.799,1059,47,16.50,20.00,4,1054.0,5
1,1000501,2016,Jeep,Compass,4,350,171,3750,1250,1.956,1059,60,17.10,21.20,3,NaN,4
2,1000502,2016,Hyundai,Creta,2,260,126,4000,1500,1.582,1059,55,19.67,24.10,4,1054.0,3
3,1000503,2016,Skoda,Superb,2,250,177,5100,1750,1.798,1059,66,14.67,23.30,3,NaN,2
4,1000504,2017,Volkswagen,Passat,4,350,174,3600,1500,1.968,1059,66,17.42,20.43,4,NaN,4
5,1000506,2017,BMW,3 Series GT,2,400,188,4000,1800,1.995,1059,57,21.76,26.32,3,NaN,3
6,1000507,2017,Audi,Q3,3,250,148,5000,1400,1.395,1059,64,16.90,21.40,3,1054.0,2


In [9]:
drive_df.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000518,8e2f7799f577423c9343d1eddbbaa2b6,2017-01-06 19:00:00,0.00,45.0,48.0,92.97,177.54,205.78,142.29,143.55,2032.75
1,1000518,8e2f7799f577423c9343d1eddbbaa2b6,2017-01-06 19:00:01,70.06,50.0,45.0,93.82,175.88,204.11,151.80,144.89,2027.28
2,1000518,8e2f7799f577423c9343d1eddbbaa2b6,2017-01-06 19:00:02,74.38,50.0,45.0,85.78,170.35,202.51,143.95,148.61,2030.11
3,1000518,8e2f7799f577423c9343d1eddbbaa2b6,2017-01-06 19:00:03,71.28,54.0,48.0,93.23,161.59,197.73,142.43,140.55,2023.94
4,1000518,8e2f7799f577423c9343d1eddbbaa2b6,2017-01-06 19:00:04,77.70,52.0,44.0,93.62,161.01,201.08,135.65,142.23,2037.57


In [10]:
print('drive shape :',drive_df.shape)
print('No. of unique trips in drive :',len(drive_df['trip_id'].unique()))
print('Minimum date : ',drive_df['datetime'].min(),' Maximum date : ',drive_df['datetime'].max())

drive shape : (9217528, 12)
No. of unique trips in drive : 1708
Minimum date :  2017-01-01 00:00:00  Maximum date :  2017-02-28 12:25:55


In [11]:
trip_df.head()

,vehicle_id,trip_id,datetime,lat,long,velocity
0,1000512,861170e5f30342c78d9b706ae908dc4f,2017-01-06 21:00:00,30.812500,-91.1875,0.00
1,1000512,861170e5f30342c78d9b706ae908dc4f,2017-01-06 21:00:01,30.812778,-91.1875,45.59
2,1000512,861170e5f30342c78d9b706ae908dc4f,2017-01-06 21:00:02,30.813056,-91.1875,46.78
3,1000512,861170e5f30342c78d9b706ae908dc4f,2017-01-06 21:00:03,30.813333,-91.1875,59.48
4,1000512,861170e5f30342c78d9b706ae908dc4f,2017-01-06 21:00:04,30.813611,-91.1875,52.71


In [12]:
print('trip shape :',trip_df.shape)
print('No. of unique trips in trip :',len(trip_df['trip_id'].unique()))
print('Minimum date : ',trip_df['datetime'].min(),' Maximum date : ',trip_df['datetime'].max())

trip shape : (9217528, 6)
No. of unique trips in trip : 1708
Minimum date :  2017-01-01 00:00:00  Maximum date :  2017-02-28 12:25:55


In [13]:
weather_df.head()

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit
0,258,38,2017-02-01,07Z,29.8125,-92.6875,286.530000,k,0.000000,kg/m^2,0.210000,m/s,2.540000,m/s
1,205,38,2017-01-15,23Z,29.8125,-99.3125,291.740000,k,0.034400,kg/m^2,-7.380000,m/s,7.190000,m/s
2,182,60,2017-01-14,04Z,32.5625,-102.1875,282.400000,k,0.000000,kg/m^2,-5.980000,m/s,-2.080000,m/s
3,124,43,2017-02-28,02Z,30.4375,-109.4375,289.970000,k,0.000000,kg/m^2,3.090000,m/s,6.350000,m/s
4,211,33,2017-01-14,09Z,29.1875,-98.5625,290.530000,k,0.000000,kg/m^2,-4.810000,m/s,0.010000,m/s


In [14]:
print('weather shape :',weather_df.shape)
print('Minimum date : ',weather_df['date'].min(),' Maximum date : ',weather_df['date'].max())
print('Unique temperature data units : ',weather_df['temperature_unit'].unique(),' Unique precipitaion data units : ',weather_df['precipitation_unit'].unique())

weather shape : (10586016, 14)
Minimum date :  2017-01-01  Maximum date :  2017-02-28
Unique temperature data units :  ['k']  Unique precipitaion data units :  ['kg/m^2']


# Merge drive and vehicle data set

In [15]:
#localizing to CST timezone as this is the timezone at latitude and longitude from dataset
drive_df['datetime'] = drive_df['datetime'].dt.tz_localize('US/Central')
#Converting to PST time zone as required
drive_df['datetime'] = drive_df['datetime'].dt.tz_convert('US/Pacific')
all_data = pd.merge(drive_df,vehicle_df,on = 'vehicle_id',how = 'left').fillna(0)
#del drive_df,vehicle_df
gc.collect()

22

# Merge with trip data set

In [16]:
#localizing to CST timezone as this is the timezone at latitude and longitude from dataset
trip_df['datetime'] = trip_df['datetime'].dt.tz_localize('US/Central')
#Converting to PST time zone as required
trip_df['datetime'] = trip_df['datetime'].dt.tz_convert('US/Pacific')
all_data = pd.merge(all_data,trip_df,on = ['datetime','trip_id','vehicle_id'],how = 'left').fillna(0)
#del trip_df
gc.collect()


54

# Merge with weather data set

In [17]:
def geohash(lat,lon):
    res = []
    for i in range(len(lat)):
        res.append(pgh.encode(lat[i], lon[i], precision=5))
    return res

In [18]:
weather_df['time'] = pd.to_datetime(weather_df['time'],format = '%HZ')
weather_df['time'] = weather_df['time'].dt.tz_localize('US/Pacific')

#Chaning the temperature data to Fahrenheit as required
weather_df['temperature_data'] = weather_df['temperature_data'].astype(np.float32)
weather_df['temperature_data'] = weather_df['temperature_data']*(9/5) - 459.67
weather_df['precipitation_data'] = weather_df['precipitation_data'].astype(np.float32)

all_data['day'] = all_data['datetime'].dt.day
all_data['month'] = all_data['datetime'].dt.month
all_data['year'] = all_data['datetime'].dt.year
all_data['hour'] = all_data['datetime'].dt.hour

weather_df['date'] = pd.to_datetime(weather_df['date'],format = '%Y-%m-%d',errors='raise')
weather_df['date'] = weather_df['date'].dt.tz_localize('US/Pacific')
weather_df['day'] = weather_df['date'].dt.day
weather_df['month'] = weather_df['date'].dt.month
weather_df['year'] = weather_df['date'].dt.year
weather_df['hour'] = weather_df['time'].dt.hour

#Getting geohash value with precision 5 for weather data as well as combined data as this will be used for merging
all_lats = all_data.lat.values
all_longs = all_data.long.values

all_data['geohash'] = geohash(all_lats,all_longs)

weather_lats = weather_df.lat.values
weather_lons = weather_df.lon.values

weather_df['geohash'] = geohash(weather_lats,weather_lons)


#Merging on hour,day,month,year as weather data is constant for complete hour
all_data = pd.merge(all_data,weather_df,on = ['hour','day','month','year','geohash'],how = 'left').fillna(0)

#del all_lats,all_longs,weather_lats,weather_lons,weather_df
gc.collect()



7

# Keep only the columns required and downcast to reduce memory size

In [19]:
all_data = all_data[['vehicle_id','trip_id','datetime','eng_load','rpm','max_torque','max_horsepower',
                     'max_horsepower_rpm','max_torque_rpm','lat_x','long','velocity_x','velocity_y','temperature_data','precipitation_data']]
gc.collect()
all_data = downcast_dtypes(all_data)
all_data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9217528 entries, 0 to 9217527
Data columns (total 15 columns):
vehicle_id            int32
trip_id               object
datetime              datetime64[ns, US/Pacific]
eng_load              float32
rpm                   float32
max_torque            float32
max_horsepower        float32
max_horsepower_rpm    float32
max_torque_rpm        float32
lat_x                 float32
long                  float32
velocity_x            float32
velocity_y            float32
temperature_data      float32
precipitation_data    float32
dtypes: datetime64[ns, US/Pacific](1), float32(12), int32(1), object(1)
memory usage: 1.3 GB


# Get week start date, torque utilization,active horsepower,horsepower utilization,rpm utilization,distance,time and acceleration fields which will be required in calculating required features

In [20]:
@njit(parallel=True)
def torque_utilization(eng_load):
    res = np.empty(eng_load.shape)
    for i in range(len(eng_load)):
        res[i] = eng_load[i]/255
    return res

@njit(parallel=True)
def active_horsepower(eng_load,max_torque,rpm):
    res = np.empty(eng_load.shape)
    for i in range(len(eng_load)):
        res[i] = (eng_load[i]/255)*max_torque[i]*(rpm[i]/5252)
    return res

@njit(parallel=True)
def horsepower_utilization(max_horsepower,active_horsepower):
    res = np.empty(max_horsepower.shape)
    for i in range(len(max_horsepower)):
        if max_horsepower[i] != 0:
            res[i] = active_horsepower[i]/max_horsepower[i]
        else:
            res[i] = 0
    return res

@njit(parallel=True)
def rpm_utilization(rpm,max_horsepower_rpm):
    res = np.empty(rpm.shape)
    for i in range(len(rpm)):
        if max_horsepower_rpm[i] != 0:
            res[i] = rpm[i]/max_horsepower_rpm[i]
        else:
            res[i] = 0
    return res

def distance(lat,lon):
    res = np.empty(lat.shape)
    res[0] = 0
    for i in range(1,len(lat)):
        res[i] = haversine((lat[i-1],lon[i-1]),(lat[i],lon[i]))
    return res

def calculate_time(datetime,datetime_min):
    res = np.empty(datetime.shape)
    res[0] = 0
    for i in range(1,len(datetime)):
        res[i] = (datetime[i]-datetime[i-1])/np.timedelta64(1,'s')
        if datetime[i] == datetime_min[i]:
            res[i] = 0
    return res

@njit(parallel=True)
def acceleration(velocity_y,time):
    res = np.empty(velocity_y.shape)
    res[0] = 0
    for i in range(1,len(velocity_y)):
        if time[i] != 0:
            res[i] = ((velocity_y[i] - velocity_y[i-1])*0.277778)/(time[i])
        else:
            res[i] = 0
    return res

@njit(parallel=True)
def update_distance(datetime,datetime_min,distance):
    res = np.empty(datetime.shape)
    for i in range(len(datetime)):
        if datetime[i] == datetime_min[i]:
            res[i] = 0
        else:
            res[i] = distance[i]
    return res

@njit(parallel=True)
def update_acceleration(datetime,datetime_min,acceleration):
    res = np.empty(datetime.shape)
    for i in range(len(datetime)):
        if datetime[i] == datetime_min[i]:
            res[i] = 0
        else:
            res[i] = acceleration[i]
    return res


In [21]:
all_data['day_of_week'] = all_data.datetime.dt.weekday

all_data['week_start_date'] = all_data.datetime - all_data.day_of_week * timedelta(days = 1)

In [22]:
# Calculating the min datetime for every veicle and trip as distance and acceleration should be set to 0 at this time
# as well as week start day should be adjusted if min datetime is after monday for this week

gb = all_data.groupby(['vehicle_id','trip_id'],as_index = False).agg({'datetime':{'datetime_min':'min'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data,gb,on =['trip_id','vehicle_id'],how = 'left').fillna(0)
del gb
gc.collect()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


91

In [23]:
all_data['week_start_date'] = np.where(all_data['datetime_min'] > all_data['week_start_date'],
                                       all_data['datetime_min'],all_data['week_start_date'])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
all_data['week_start_date'] = all_data.week_start_date.dt.date

all_data = all_data.sort_values(['vehicle_id','trip_id','datetime'],ascending = [True,True,True])

all_data['torque_utilization'] = torque_utilization(all_data['eng_load'].values)

all_data['active_horsepower'] = active_horsepower(all_data['eng_load'].values,all_data['max_torque'].values,all_data['rpm'].values)

all_data['horsepower_utilization'] = horsepower_utilization(all_data['max_horsepower'].values,all_data['active_horsepower'].values)

all_data['rpm_utilization'] = rpm_utilization(all_data['rpm'].values,all_data['max_horsepower_rpm'].values)



all_data['distance'] = distance(all_data['lat_x'].values,all_data['long'].values)

all_data['time'] = calculate_time(all_data['datetime'].values,all_data['datetime_min'].values)

all_data['acceleration'] = acceleration(all_data['velocity_y'].values,all_data['time'].values)

all_data['distance'] = update_distance(all_data['datetime'].values,all_data['datetime_min'].values,all_data['distance'].values)

all_data['acceleration'] = update_distance(all_data['datetime'].values,all_data['datetime_min'].values,all_data['acceleration'].values)

all_data = downcast_dtypes(all_data)


In [25]:
# all_data.dtypes

# Get engine features

In [26]:
def get_engine_features(group):
    #print(type(group))
    ft_torque_util_60pct_s = group.loc[(group['torque_utilization'] >= 0.6) & (group['torque_utilization'] < 0.7),'time'].sum()
    ft_torque_util_70pct_s = group.loc[(group['torque_utilization'] >= 0.7) & (group['torque_utilization'] < 0.8),'time'].sum()
    ft_torque_util_80pct_s = group.loc[(group['torque_utilization'] >= 0.8) & (group['torque_utilization'] < 0.9),'time'].sum()
    ft_torque_util_90pct_s = group.loc[(group['torque_utilization'] >= 0.9),'time'].sum()
    ft_horsepower_util_50pct_s = group.loc[(group['horsepower_utilization'] >= 0.5) & (group['horsepower_utilization'] < 0.6),
                                           'time'].sum()
    ft_horsepower_util_60pct_s = group.loc[(group['horsepower_utilization'] >= 0.6) & (group['horsepower_utilization'] < 0.7),
                                           'time'].sum()
    
    ft_horsepower_util_70pct_s = group.loc[(group['horsepower_utilization'] >= 0.7) & (group['horsepower_utilization'] < 0.8),
                                           'time'].sum()
    
    ft_torque_util_80pct_s = group.loc[(group['horsepower_utilization'] >= 0.8) & (group['horsepower_utilization'] < 0.9),
                                           'time'].sum()
    
    ft_rpm_util_50pct_s = group.loc[(group['rpm_utilization'] >= 0.5) & (group['rpm_utilization'] < 0.6),
                                           'time'].sum()
    
    ft_rpm_util_60pct_s = group.loc[(group['rpm_utilization'] >= 0.6) & (group['rpm_utilization'] < 0.7),
                                           'time'].sum()
    cols = ['ft_torque_util_60pct_s','ft_torque_util_70pct_s','ft_torque_util_80pct_s','ft_torque_util_90pct_s',
            'ft_horsepower_util_50pct_s','ft_horsepower_util_60pct_s','ft_horsepower_util_70pct_s','ft_torque_util_80pct_s',
            'ft_rpm_util_50pct_s','ft_rpm_util_60pct_s']
    return pd.Series([ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,
            ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_torque_util_80pct_s,
            ft_rpm_util_50pct_s,ft_rpm_util_60pct_s],index = cols)
    


In [27]:
engine_features = all_data.groupby(['vehicle_id','week_start_date']).apply(get_engine_features).reset_index()
engine_features.columns = ef.columns
engine_features = engine_features.sort_values(['vehicle_id','week_start_date'],ascending = [True,True])
#engine_features = round_floats(engine_features)
engine_features.to_csv('engine_features.csv',index = False)
#del engine_features
gc.collect()

6397

In [33]:
# engine_features.shape

(516, 12)

# Get drive features

In [28]:
drive_features = None
temp = all_data.groupby(['trip_id'])
values = []
for name,group in temp:
    #print(type(name))
    #print(name)
    ft_cnt_vehicle_deaccel_val = 0
    ft_sum_time_accel_val = 0
    ft_sum_time_deaccel_val = 0
    ft_cnt_vehicle_accel_val = 0
    ft_sum_hard_brakes_10_flg_val = 0
    ft_sum_hard_brakes_3_flg_val = 0
    ft_sum_hard_accel_10_flg_val = 0
    ft_sum_hard_accel_3_flg_val = 0
    min_accl = 0
    max_accl = 0
    first_index = None
    #indexes = group.index.values
    #print(indexes)
    accelerations = group['acceleration'].values
    times = group['time'].values
    
    for i in range(len(accelerations)):
         
        if accelerations[i] < 0:
            
            ft_sum_time_deaccel_val = ft_sum_time_deaccel_val + times[i]
            
            if i == 0:
                ft_cnt_vehicle_deaccel_val = ft_cnt_vehicle_deaccel_val + 1
                min_accl = accelerations[i]
            elif accelerations[i-1] > 0:
                ft_cnt_vehicle_deaccel_val = ft_cnt_vehicle_deaccel_val + 1
                min_accl = accelerations[i]
                
            if (i+1) < len(accelerations) and accelerations[i + 1] > 0 :
                min_accl = min(min_accl,accelerations[i])
                if min_accl <= -10:
                    ft_sum_hard_brakes_10_flg_val = ft_sum_hard_brakes_10_flg_val +1
                if min_accl <= -3 and min_accl > -10:
                    ft_sum_hard_brakes_3_flg_val = ft_sum_hard_brakes_3_flg_val + 1
            elif (i+1) == len(accelerations):
                min_accl = min(min_accl,accelerations[i])
                if min_accl <= -10:
                    ft_sum_hard_brakes_10_flg_val = ft_sum_hard_brakes_10_flg_val +1
                if min_accl <= -3 and min_accl > -10:
                    ft_sum_hard_brakes_3_flg_val = ft_sum_hard_brakes_3_flg_val + 1
            else:
                min_accl = min(min_accl,accelerations[i])

        if accelerations[i] > 0:
            ft_sum_time_accel_val = ft_sum_time_accel_val + times[i]
            
            if i == 0:
                ft_cnt_vehicle_accel_val = ft_cnt_vehicle_accel_val + 1
                max_accl = accelerations[i]
            elif accelerations[i-1] < 0:
                ft_cnt_vehicle_accel_val = ft_cnt_vehicle_accel_val + 1
                max_accl = accelerations[i]
                
            if (i+1) < len(accelerations) and accelerations[i + 1] < 0 :
                max_accl = max(max_accl,accelerations[i])
                if max_accl >= 10:
                    ft_sum_hard_accel_10_flg_val = ft_sum_hard_accel_10_flg_val +1
                if max_accl >= 3 and max_accl < 10:
                    ft_sum_hard_accel_3_flg_val = ft_sum_hard_accel_3_flg_val + 1
            elif (i+1) == len(accelerations):
                max_accl = max(max_accl,accelerations[i])
                if max_accl >= 10:
                    ft_sum_hard_accel_10_flg_val = ft_sum_hard_accel_10_flg_val +1
                if max_accl >= 3 and max_accl < 10:
                    ft_sum_hard_accel_3_flg_val = ft_sum_hard_accel_3_flg_val + 1
            else:
                max_accl = max(max_accl,accelerations[i])
            

    values.append(np.array([name,ft_cnt_vehicle_deaccel_val, ft_sum_time_accel_val,
        ft_sum_time_deaccel_val, ft_cnt_vehicle_accel_val,
       ft_sum_hard_brakes_10_flg_val, ft_sum_hard_brakes_3_flg_val,
       ft_sum_hard_accel_10_flg_val, ft_sum_hard_accel_3_flg_val
                      ]))
    

del accelerations,times,temp,group
gc.collect()

drive_features = pd.DataFrame(values,columns = df.columns)

del values
gc.collect()

drive_features[['ft_cnt_vehicle_deaccel_val', 'ft_sum_hard_brakes_10_flg_val',
       'ft_sum_hard_brakes_3_flg_val', 'ft_sum_time_deaccel_val','ft_cnt_vehicle_accel_val',
        'ft_sum_hard_accel_10_flg_val','ft_sum_hard_accel_3_flg_val','ft_sum_time_accel_val']] = drive_features[[
    'ft_cnt_vehicle_deaccel_val', 'ft_sum_hard_brakes_10_flg_val',
       'ft_sum_hard_brakes_3_flg_val', 'ft_sum_time_deaccel_val','ft_cnt_vehicle_accel_val',
        'ft_sum_hard_accel_10_flg_val','ft_sum_hard_accel_3_flg_val','ft_sum_time_accel_val']].astype(np.float32)

drive_features = drive_features[['trip_id','ft_cnt_vehicle_deaccel_val', 'ft_sum_hard_brakes_10_flg_val',
       'ft_sum_hard_brakes_3_flg_val', 'ft_sum_time_deaccel_val','ft_cnt_vehicle_accel_val',
        'ft_sum_hard_accel_10_flg_val','ft_sum_hard_accel_3_flg_val','ft_sum_time_accel_val']]

#drive_features = drive_features.drop(drive_features[drive_features.sum(axis = 1) == 0].index)
#drive_features = round_floats(drive_features)
drive_features = drive_features.sort_values(['trip_id'],ascending = [True])
drive_features.to_csv('drive_features.csv',index = False)
#del drive_features
gc.collect()

7

In [34]:
# drive_features.shape

(1708, 9)

# Get weather features

In [29]:
def get_weather_features(group):
    total_light_rain_driving_km = group.loc[(group['precipitation_data'] > 0) & (group['precipitation_data'] <= 2.5) & (group['temperature_data'] > 32),'distance'].sum()
    total_light_freezing_rain_driving_km = group.loc[(group['precipitation_data'] > 0) & (group['precipitation_data'] <= 2.5) & (group['temperature_data'] > 27)  
                                                     & (group['temperature_data'] <= 32),'distance'].sum()
    total_light_snow_driving_km = group.loc[(group['precipitation_data'] > 0) & (group['precipitation_data'] <= 2.5) & (group['temperature_data'] <= 27),'distance'].sum()
    total_moderate_rain_driving_km = group.loc[(group['precipitation_data'] > 2.5) & (group['precipitation_data'] <= 7.6) & (group['temperature_data'] > 32),'distance'].sum()
    total_moderate_freezing_rain_driving_km = group.loc[(group['precipitation_data'] > 2.5) & (group['precipitation_data'] <= 7.6) & (group['temperature_data'] > 27)  
                                                     & (group['temperature_data'] <= 32),'distance'].sum()
    total_moderate_snow_driving_km = group.loc[(group['precipitation_data'] > 2.5) & (group['precipitation_data'] <= 7.6) & (group['temperature_data'] <= 27),'distance'].sum()
    total_heavy_rain_driving_km = group.loc[(group['precipitation_data'] > 7.6) & (group['temperature_data'] > 32) ,'distance'].sum()
    cols = ['total_light_rain_driving_km','total_light_freezing_rain_driving_km', 'total_light_snow_driving_km','total_moderate_rain_driving_km',
            'total_moderate_freezing_rain_driving_km','total_moderate_snow_driving_km', 'total_heavy_rain_driving_km']
    return pd.Series([total_light_rain_driving_km,total_light_freezing_rain_driving_km, total_light_snow_driving_km,total_moderate_rain_driving_km,total_moderate_freezing_rain_driving_km,
       total_moderate_snow_driving_km, total_heavy_rain_driving_km],index = cols)
    

In [30]:
weather_features = all_data.groupby(['vehicle_id','week_start_date']).apply(get_weather_features).reset_index()
weather_features.columns = wf.columns
weather_features = weather_features.sort_values(['vehicle_id','week_start_date'],ascending = [True,True])
#weather_features = round_floats(weather_features)
weather_features.to_csv('weather_features.csv',index = False)
#del weather_features,all_data,df,ef,wf
gc.collect()

50

In [35]:
# weather_features.shape

(516, 9)

In [31]:
end_time = time.time()
print('total time elapsed %f seconds'%(end_time - start_time))

total time elapsed 821.786804 seconds
